In [ ]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
import pandas as pd

In [ ]:
def display_image(img: np.ndarray, title: str=""):
    '''
    Display an opencv/numpy image.

    Args:
        img: np.ndarray, the image
        title: str, title to display on the plot
    '''
    plt.imshow(img[...,::-1])
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
# define constants

# OpenCV example
#SCENE_IMAGE_PATH = 'box_in_scene.png'
#QUERY_IMAGE_PATH = 'box.png'

# Shell logo from openlogo
#SCENE_IMAGE_PATH = '/data/openlogo/JPEGImages/shellimg000093.jpg'
#QUERY_IMAGE_PATH = '/data/openlogo/JPEGImages/shellimg000130.jpg'

# Oxford Buildings
#  - Hertford College
#SCENE_IMAGE_PATH = '/data/oxford-buildings/oxbuild_images/hertford_000034.jpg'
#QUERY_IMAGE_PATH = '/data/oxford-buildings/oxbuild_images/hertford_000027.jpg'

#  - Tower of the Five Orders
#SCENE_IMAGE_PATH = '/data/oxford-buildings/oxbuild_images/bodleian_000037.jpg'
#SCENE_IMAGE_PATH = '/data/oxford-buildings/oxbuild_images/bodleian_000000.jpg'
#QUERY_IMAGE_PATH = '/data/oxford-buildings/oxbuild_images/bodleian_000041.jpg' # Fail

MAX_SCENE_FEATURES = 10000
MAX_QUERY_FEATURES = 10000

In [ ]:
scene = cv2.imread(SCENE_IMAGE_PATH)
display_image(scene, 'scene image')

In [ ]:
scene_gray = cv2.cvtColor(scene, cv2.COLOR_BGR2GRAY)

In [ ]:
# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_shi_tomasi/py_shi_tomasi.html
corners = cv2.goodFeaturesToTrack(scene_gray,maxCorners=2000,qualityLevel=0.01,minDistance=5)
corners = np.int0(corners)
display_img = scene.copy()

# corner features are positional only
# relies on optical flow to understand surroundings
x, y = corners[0].ravel()
print(f"corners[0] = {corners[0]}, value = {scene_gray[x,y]}")
print(pd.DataFrame(data=scene_gray[x-2:x+3,y-2:y+3], index=range(y-2,y+3), columns=range(x-2,x+3)))

for i in corners:
    x,y = i.ravel()
    cv2.circle(display_img,(x,y),5,[0,255,0],-1)

display_image(display_img, 'good features')

In [ ]:
# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_orb/py_orb.html
orb = cv2.ORB_create(nfeatures=MAX_SCENE_FEATURES)
scene_kp, scene_des = orb.detectAndCompute(scene, None)

print(f"#kp={len(scene_kp)},#des={len(scene_des)}")
print(f"{scene_kp[0].pt} {scene_kp[0].angle} {scene_kp[0].size}")
print(f"len(des)={len(scene_des[0])}, des[0]={scene_des[0]}")

# SIFT features would be better in production
# also increases the descriptor dimensionality to 128
# which means better matching

In [ ]:
kp_img = cv2.drawKeypoints(scene, scene_kp, None, color=(0, 255, 0), flags=0)
display_image(kp_img, "ORB features on scene image")

In [ ]:
# Query By Example (QBE) aka Reverse Image Search
query_img = cv2.imread(QUERY_IMAGE_PATH)
display_image(query_img, 'query image')

In [ ]:
query_gray = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)

In [ ]:
query_orb = cv2.ORB_create(nfeatures=MAX_QUERY_FEATURES)
query_kp, query_des = query_orb.detectAndCompute(query_gray, None)

print(f"#kp={len(query_kp)},#des={len(query_des)}")
print(f"{query_kp[0].pt} {query_kp[0].angle} {query_kp[0].size}")
print(f"len(des)={len(query_des[0])}, des[0]={query_des[0]}")

In [ ]:
kp_img = cv2.drawKeypoints(query_img, query_kp, None, color=(0, 255, 0), flags=0)
display_image(kp_img, "ORB features on query image")

In [ ]:
# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_matcher/py_matcher.html
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(query_des, scene_des)

sorted_matches = sorted(matches, key=lambda x: x.distance)

def display_match_descriptors(query_descriptors, scene_descriptors, sorted_matches, index):
    print(f"query des: {query_descriptors[sorted_matches[index].queryIdx]}")
    print(f"scene des: {scene_descriptors[sorted_matches[index].trainIdx]}")
    print(f"hamming distance: {sorted_matches[index].distance}")
    print("\n")

display_match_descriptors(query_des, scene_des, sorted_matches, 0)
display_match_descriptors(query_des, scene_des, sorted_matches, 1)

In [ ]:
match_img = cv2.drawMatches(query_img,query_kp,scene,scene_kp,sorted_matches[:50],None,flags=2)
display_image(match_img, 'brute force feature matching')

In [ ]:
# solve transform from query image to scene image
dst_pts = np.float32([ scene_kp[m.trainIdx].pt for m in sorted_matches ]).reshape(-1,1,2)
src_pts = np.float32([ query_kp[m.queryIdx].pt for m in sorted_matches ]).reshape(-1,1,2)

M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
np.set_printoptions(suppress=True)
print(f"perspective transform = \n{M}")

In [ ]:
# transform query image to scene image and blend
warp = cv2.warpPerspective(query_img, M, (scene.shape[1], scene.shape[0]))
blended = cv2.addWeighted(scene, 0.2, warp, 0.8, 0)
display_image(blended, 'warped query image into scene')
display_image(scene, 'original scene')